## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Meshcherino,RU,2021-01-07 21:30:43,55.1957,38.3600,23.00,86,90,6.71,mist
1,1,Butaritari,KI,2021-01-07 21:30:43,3.0707,172.7902,81.52,83,9,20.04,light rain
2,2,Mataura,NZ,2021-01-07 21:30:43,-46.1927,168.8643,57.00,74,40,1.99,scattered clouds
3,3,Tuktoyaktuk,CA,2021-01-07 21:30:44,69.4541,-133.0374,-11.20,77,20,9.22,few clouds
4,4,Saint George,US,2021-01-07 21:30:44,37.1041,-113.5841,57.00,27,1,0.98,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Butaritari,KI,2021-01-07 21:30:43,3.0707,172.7902,81.52,83,9,20.04,light rain
7,7,Olinda,BR,2021-01-07 21:30:44,-8.0089,-34.8553,80.60,69,40,9.22,scattered clouds
8,8,Arraial Do Cabo,BR,2021-01-07 21:30:45,-22.9661,-42.0278,84.20,74,0,9.22,clear sky
10,10,Rikitea,PF,2021-01-07 21:30:45,-23.1203,-134.9692,78.33,70,100,6.53,overcast clouds
16,16,Kapaa,US,2021-01-07 21:30:47,22.0752,-159.3190,78.80,73,20,9.22,few clouds
17,17,Andros Town,BS,2021-01-07 21:30:47,24.7000,-77.7667,75.20,57,20,10.36,few clouds
18,18,Carnarvon,AU,2021-01-07 21:30:47,-24.8667,113.6333,82.40,69,40,11.50,scattered clouds
21,21,Puerto Escondido,MX,2021-01-07 21:30:48,15.8500,-97.0667,82.40,65,40,10.36,light rain
23,23,Georgetown,MY,2021-01-07 21:30:05,5.4112,100.3354,79.00,83,20,6.08,few clouds
24,24,Portobelo,PA,2021-01-07 21:30:49,9.5500,-79.6500,87.01,99,54,8.72,broken clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.52,light rain,3.0707,172.7902,
7,Olinda,BR,80.60,scattered clouds,-8.0089,-34.8553,
8,Arraial Do Cabo,BR,84.20,clear sky,-22.9661,-42.0278,
10,Rikitea,PF,78.33,overcast clouds,-23.1203,-134.9692,
16,Kapaa,US,78.80,few clouds,22.0752,-159.3190,
17,Andros Town,BS,75.20,few clouds,24.7000,-77.7667,
18,Carnarvon,AU,82.40,scattered clouds,-24.8667,113.6333,
21,Puerto Escondido,MX,82.40,light rain,15.8500,-97.0667,
23,Georgetown,MY,79.00,few clouds,5.4112,100.3354,
24,Portobelo,PA,87.01,broken clouds,9.5500,-79.6500,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("done")
      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
done


In [65]:
hotel_df.index

for i in hotel_df.index:
    if hotel_df["City"][i] == "":
        hotel_df.drop[i]
        print("row"+ i +"Dropped")

In [67]:
# 7. Drop the rows where there is no Hotel Name.
for i in hotel_df.index:
    if hotel_df["City"][i] == "":
        hotel_df.drop[i]
        print("row"+ i +"Dropped")
clean_hotel_df = hotel_df

In [68]:
# 8a. Create the output File (CSV)
output_data_file = "clean_hotel_data.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [69]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [71]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))